In [ ]:
from qTools import *
import Cahit as ck

import scipy.sparse as sp
import numpy as np

from functools import partial
from multiprocessing import Pool, cpu_count

import matplotlib.pyplot as plt
import datetime

import copy

In [ ]:
resonatorDimension = 200
g = 1.79
qfreq = 0
resFreq = 2

In [ ]:
qSim = Simulation()
qSim.finalTime = 1.2
qSim.stepSize = 0.02
JCSys = qSim.qSys
#qSim.delState = True

In [ ]:
cav = Cavity(dimension=resonatorDimension, frequency=resFreq)
JCSys.addSubSys(cav)

qub = JCSys.createSubSys(subClass=Qubit)
qub.frequency = qfreq

In [ ]:
digitalRabi = qProtocol(superSys=JCSys)
JCunitary = freeEvolution(ratio=0.5, superSys=JCSys)
bitFlip = xGate(implementation='instant', superSys=qub, fixed=True)
digitalRabi.addStep(JCunitary, bitFlip, JCunitary, JCunitary, bitFlip, JCunitary)

JCSys.unitary = digitalRabi

In [ ]:
JCcoupling = JCSys.JC(cav, qub, g)

In [ ]:
RabiSys = JCSys.copy(name='Rabi')
RabiCoupling = RabiSys.Rabi(RabiSys.qSystems['Cavity2'],RabiSys.qSystems['Qubit2'], g)
qSim.subSystems = RabiSys

In [ ]:
JCSys.initialState = [0,0]

cavFreqSweep = qSim.Loop.addSweep(sys=cav, sweepKey='frequency')
cavFreqSweep.sweepList = np.arange(-4, 4+0.05, 0.05)

"""gSweep = qSim.beforeLoop.addSweep(sys=JCcoupling, sweepKey='couplingStrength')
gSweep.sweepList = np.arange(0, 2, 0.05)"""

In [ ]:
# TODO copy should copy the initial state 
RabiSys.initialState = [0,0]

cavFreqSweep2 = qSim.Loop.addSweep(sys=RabiSys.qSystems['Cavity2'], sweepKey='frequency')
cavFreqSweep2.sweepList = np.arange(-8, 8+0.1, 0.1)

In [ ]:
cavParity = parityEXP(cav.freeMat)
qubPop = qub.freeMat

def expect(qSim, state1, state2):
    qSim.qRes.result = expectationKet(cavParity, state1)
    qSim.qRes.result = fidelityKet(state1, state2)

qSim.compute = expect

In [ ]:
nw = datetime.datetime.now()

results = qSim.run(p=True, coreCount='all')

en = datetime.datetime.now()

print(en-nw)

In [ ]:
def plotRes(xList = cavFreqSweep2.sweepList):
    fig = plt.figure(figsize=(12,9))
    setC = [0.88, 0.25, 0.02, 0.6]
    axGrids = ck.Plotting.Functions.grid(2, 1, fig=fig)

    ck.Plotting.SimplePlots.colorPlot(xList, [x*qSim.stepSize for x in range(qSim.steps+1)], results.results[0],ax=axGrids[0], gif=fig, setC=setC)
    ck.Plotting.SimplePlots.colorPlot(xList, [x*qSim.stepSize for x in range(qSim.steps+1)], results.results[1],ax=axGrids[1], gif=fig, setC=setC)
plotRes()

In [ ]:
JCSys.reset()
JCSys.createSysCoupling([qub,cav],[QuantumToolbox.operators.sigmax, QuantumToolbox.operators.create], couplingStrength=g)
JCSys.createSysCoupling([cav, qub],[QuantumToolbox.operators.destroy, QuantumToolbox.operators.sigmax], couplingStrength=g)
JCSys.couplingName = 'Rabi'

In [ ]:
# TODO remove corresponding sweeps
qSim.removeSys(RabiSys)

def expectn(qSim, state1):
    qSim.qRes.result = expectationKet(cavParity, state1)
    qSim.qRes.result = fidelityKet(qSim.qSys.initialState, state1)

qSim.compute = expectn

In [ ]:
#p1 = Pool(processes=cpu_count())
nw = datetime.datetime.now()

results = qSim.run()

en = datetime.datetime.now()
print(en-nw)

In [ ]:
fig = plt.figure(figsize=(12,9))
setC = [0.88, 0.25, 0.02, 0.6]
axGrids = ck.Plotting.Functions.grid(2, 1, fig=fig)

ck.Plotting.SimplePlots.colorPlot(cavFreqSweep.sweepList, [x*qSim.stepSize for x in range(qSim.steps+1)], results.results[0],ax=axGrids[0], gif=fig, setC=setC)
ck.Plotting.SimplePlots.colorPlot(cavFreqSweep.sweepList, [x*qSim.stepSize for x in range(qSim.steps+1)], results.results[1],ax=axGrids[1], gif=fig, setC=setC)

In [ ]:
JCSys.reset(to='JC')

nw = datetime.datetime.now()

results = qSim.run()

en = datetime.datetime.now()
plotRes(cavFreqSweep.sweepList)

In [ ]:
JCSys.reset(to='JC')

cavFreqSweep.sweepList = np.arange(-2, 2+0.025, 0.025)
JCSys.unitary = None

In [ ]:
nw = datetime.datetime.now()

results = qSim.run()

en = datetime.datetime.now()
plotRes(cavFreqSweep.sweepList)

In [ ]:
JCcoupling.couplingStrength = 1
nw = datetime.datetime.now()

results = qSim.run()

en = datetime.datetime.now()
print(en-nw)
plotRes(cavFreqSweep.sweepList)